# Introduction to Regression

## Getting the data

Here im using the melbourne housing dataset from kaggle. https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot

In [1]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
import pandas as pd
from kagglehub import KaggleDatasetAdapter
from statsmodels.regression.linear_model import OLS

# Set the path to the file you'd like to load
file_path = "melb_data.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "dansbecker/melbourne-housing-snapshot",
  file_path,
  pandas_kwargs={'encoding': 'latin1'}
)
df

C:\Users\zak\Projects\PyCharmProjects\data-science\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


## Target

TODO: Check the value is correct for the data

Aiming for less than 15,000 rmse on house price prediction

## Missing Data

In [2]:
# "Handle" missing data
df.dropna(inplace=True)
df.isnull().sum()
df.isnull().mean()


Suburb           0.0
Address          0.0
Rooms            0.0
Type             0.0
Price            0.0
Method           0.0
SellerG          0.0
Date             0.0
Distance         0.0
Postcode         0.0
Bedroom2         0.0
Bathroom         0.0
Car              0.0
Landsize         0.0
BuildingArea     0.0
YearBuilt        0.0
CouncilArea      0.0
Lattitude        0.0
Longtitude       0.0
Regionname       0.0
Propertycount    0.0
dtype: float64

## Non-numerical data

In [3]:
# Create features and target
y = df.Price
X = df.select_dtypes("number").drop(columns="Price")
X

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.00,1900.0,-37.80790,144.99340,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.00,1900.0,-37.80930,144.99440,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.00,2014.0,-37.80720,144.99410,4019.0
6,3,2.5,3067.0,4.0,2.0,0.0,245.0,210.00,1910.0,-37.80240,144.99930,4019.0
7,2,2.5,3067.0,2.0,1.0,2.0,256.0,107.00,1890.0,-37.80600,144.99540,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12205,3,35.5,3757.0,3.0,2.0,1.0,972.0,149.00,1996.0,-37.51232,145.13282,2170.0
12206,3,6.8,3016.0,3.0,1.0,0.0,179.0,115.00,1890.0,-37.86558,144.90474,6380.0
12207,1,6.8,3016.0,1.0,1.0,1.0,0.0,35.64,1967.0,-37.85588,144.89936,6380.0
12209,2,4.6,3181.0,2.0,1.0,1.0,0.0,61.60,2012.0,-37.85581,144.99025,4380.0


In [4]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

## Create train test split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Stage 1: Training
We fit the model using only the training data.

In [6]:
simple_linear_model = LinearRegression()
# simple_linear_model.fit(X_train, y_train)
# simple_linear_model.coef_

## Stage 2: Model Diagnosis (Cross-Validation)
To evaluate correctly and diagnose our model, we use Cross-Validation on our **training data** only. This allows us to keep the test set truly unseen for the final evaluation.

#### Does `cross_validate` train a better model?
No. `cross_validate` doesn't return a model at all. Its purpose is **evaluation**, not training a final model.

Think of it like this:
- **Stage 1 (Training):** You build a model using all your training data. This is the model you will actually use for predictions.
- **Stage 2 (Cross-Validation):** You "stress-test" the model's *configuration* (like the algorithm or hyperparameters). By training and testing on multiple different subsets of your data, you get a much more reliable estimate of how well that type of model works.

**Crucially**, because CV uses smaller subsets of data (e.g., 80% if using 5 folds), the internal models it trains are actually *slightly worse* than the model trained on 100% of the training data in Stage 1. We use CV to **decide** if our Stage 1 model is good or if we need to change it (e.g., try a different algorithm or add regularization).

In `cross_validate` results:
- `train_score`: Performance on the training folds.
- `test_score`: This is the **CV Score** (performance on the validation folds). It is **NOT** the final test score.

Interpretations:
- **High Variance (Overfitting)**: Train score is much better than CV score.
- **High Bias (Underfitting)**: Both scores are poor.

#### Bias and Variance
Where J is the cost function $$ J(w, b) = \frac{1}{m} \sum_{i=1}^m (f_w(x^{(i)}) - y^{(i)})^2 $$

High Bias (underfit) = J<sub>train</sub> is high and J<sub>CV</sub> is high

High Variance (overfit) = J<sub>train</sub> is low J<sub>CV</sub> is high

In [7]:
from sklearn.metrics import root_mean_squared_error
# We perform CV on the training set to diagnose the model
cv_results = cross_validate(
    simple_linear_model,
    X_train, 
    y_train, 
    scoring="neg_root_mean_squared_error", 
    cv=5, 
    return_train_score=True
)

# Convert negative RMSE to positive
train_rmse = -cv_results['train_score'].mean()
cv_rmse = -cv_results['test_score'].mean()
train_rmse_std = -cv_results['train_score'].std()
cv_rmse_std = -cv_results['test_score'].std()


# plus or minus the standard deviation

print(f"Mean Train RMSE: {train_rmse:.2f} {-cv_results['train_score']} {-train_rmse_std}")
print(f"Mean CV RMSE (Validation Score): {cv_rmse:.2f} {-cv_results['test_score']} {-cv_rmse_std}")

Mean Train RMSE: 395924.31 [398145.50015959 378870.50427711 406182.59289324 403670.21404074
 392752.74701023] 9706.247312772486
Mean CV RMSE (Validation Score): 399859.57 [393905.22597103 463654.25153988 359130.0912853  368852.46534921
 413755.79527121] 37198.525932016404


### Stage 3: Final Evaluation (The Test Set)
**The Golden Rule:** You only check your model on the test data at the **very end**, once you are happy with your model's performance in Cross-Validation.

Test is a generalization score. You get what you get on the test. If you change your model based on the test score, you are "fitting to the test set" and your results will be biased!

![Cross Validation](../../images/grid_search_cross_validation.png)

In [8]:
from sklearn.metrics import r2_score #, mean_absolute_error
import seaborn as sns
import plotly.express as px

# Final predictions on the unseen test set


simple_linear_model.fit(X_train, y_train)
y_pred = simple_linear_model.predict(X_test)

print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE: {root_mean_squared_error(y_test, y_pred):.2f}")

fig = px.scatter(x=y_pred, y=y_test, labels={'x': 'Predicted Price', 'y': 'Actual Price'}, title='Predicted vs Actual Prices')
fig.show()

ModuleNotFoundError: No module named 'plotly'

In [9]:
import statsmodels.api as sm

X_train_ols = X_train.copy()
X_test_ols = X_test.copy()
X_train_ols['constant'] = 1
X_test_ols['constant'] = 1

ols = sm.OLS(y_train, X_train_ols)
ols = ols.fit()
ols.params

Rooms            1.588458e+05
Distance        -4.165807e+04
Postcode         7.228422e+02
Bedroom2         1.470199e+04
Bathroom         1.872375e+05
Car              6.622129e+04
Landsize         1.263107e+01
BuildingArea     2.241983e+03
YearBuilt       -4.235124e+03
Lattitude       -1.126201e+06
Longtitude       7.774263e+05
Propertycount   -2.185244e+00
constant        -1.489684e+08
dtype: float64

In [10]:
# Evaluate OLS on test data


y_pred_ols = ols.predict(X_test_ols)

print(f"OLS R2 Score: {r2_score(y_test, y_pred_ols):.4f}")
print(f"OLS RMSE: {root_mean_squared_error(y_test, y_pred_ols):.2f}")

# Plot OLS Results
fig_ols = px.scatter(x=y_pred_ols, y=y_test, labels={'x': 'Predicted Price', 'y': 'Actual Price'}, title='Predicted vs Actual Prices')
fig_ols.show()

OLS R2 Score: 0.5260
OLS RMSE: 508007.10


NameError: name 'px' is not defined

## Stage 4: Polynomial Features
We can create new features by taking combinations of existing ones. A 2nd order polynomial includes the original features, their squares, and their interactions.

In [11]:
# Create polynomial features (degree 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_array = poly.fit_transform(X)

# Create a DataFrame with the new features, preserving the original index
X_poly = pd.DataFrame(
    X_poly_array, 
    columns=poly.get_feature_names_out(X.columns), 
    index=X.index
)

print(f"Original feature count: {X.shape[1]}")
print(f"Polynomial feature count: {X_poly.shape[1]}")
X_poly

Original feature count: 12
Polynomial feature count: 90


,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,...,YearBuilt^2,YearBuilt Lattitude,YearBuilt Longtitude,YearBuilt Propertycount,Lattitude^2,Lattitude Longtitude,Lattitude Propertycount,Longtitude^2,Longtitude Propertycount,Propertycount^2
1,2.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.00,1900.0,-37.80790,...,3610000.0,-71835.01000,275487.46000,7636100.0,1429.437302,-5481.895968,-151949.95010,21023.086044,582728.47460,16152361.0
2,3.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.00,1900.0,-37.80930,...,3610000.0,-71837.67000,275489.36000,7636100.0,1429.543166,-5482.136768,-151955.57670,21023.376031,582732.49360,16152361.0
4,4.0,2.5,3067.0,3.0,1.0,2.0,120.0,142.00,2014.0,-37.80720,...,4056196.0,-76143.70080,292018.11740,8094266.0,1429.384372,-5481.820938,-151947.13680,21023.289035,582731.28790,16152361.0
6,3.0,2.5,3067.0,4.0,2.0,0.0,245.0,210.00,1910.0,-37.80240,...,3648100.0,-72202.58400,276948.66300,7676290.0,1429.021446,-5481.321538,-151927.84560,21024.797000,582752.18670,16152361.0
7,2.0,2.5,3067.0,2.0,1.0,2.0,256.0,107.00,1890.0,-37.80600,...,3572100.0,-71453.34000,274041.30600,7595910.0,1429.293636,-5481.696092,-151942.31400,21023.666021,582736.51260,16152361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12205,3.0,35.5,3757.0,3.0,2.0,1.0,972.0,149.00,1996.0,-37.51232,...,3984016.0,-74874.59072,289685.10872,4331320.0,1407.174152,-5444.268786,-81401.73440,21063.535441,314938.21940,4708900.0
12206,3.0,6.8,3016.0,3.0,1.0,0.0,179.0,115.00,1890.0,-37.86558,...,3572100.0,-71565.94620,273869.95860,12058200.0,1433.802149,-5486.902025,-241582.40040,20997.383674,924492.24120,40704400.0
12207,1.0,6.8,3016.0,1.0,1.0,1.0,0.0,35.64,1967.0,-37.85588,...,3869089.0,-74462.51596,285017.04112,12549460.0,1433.067651,-5485.292784,-241520.51440,20995.824528,924457.91680,40704400.0
12209,2.0,4.6,3181.0,2.0,1.0,1.0,0.0,61.60,2012.0,-37.85581,...,4048144.0,-76165.88972,291720.38300,8812560.0,1433.062351,-5488.723356,-165808.44780,21022.172595,635057.29500,19184400.0


In [12]:
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y, random_state=0)

In [13]:
ols_poly = sm.OLS(y_train, X_train_poly)
ols_poly = ols_poly.fit()
# The summary() method is the "benchmark" feature of statsmodels, 
# providing detailed statistics (p-values, R-squared, etc.) that sklearn does not.
ols_poly.summary()

# Null hypothesis is that every coefficient is 0 i.e. it has no effect on the data
# For example, if a p value is high, we can't reject the null hypothesis. We can't reject that there is no relation.




<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     176.9
Date:                Sun, 15 Feb 2026   Prob (F-statistic):               0.00
Time:                        15:37:46   Log-Likelihood:                -65340.
No. Observations:                4647   AIC:                         1.309e+05
Df Residuals:                    4557   BIC:                         1.314e+05
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Rooms                        5.99e+07   5.06e+07      1.184      0.237   -3.93e+07    1.59e+08
Distance                    8.858e+06   1.62e+06      5.467      0.000    5.68e+06     1.2e+07
Postcode                   -3.422e+05   1.05e+05     -3.263      0.001   -5.48e+05   -1.37e+05
Bedroom2                   -6.866e+07   5.04e+07     -1.363      0.173   -1.67e+08    3.01e+07
Bathroom                   -1.131e+07   1.74e+07     -0.649      0.516   -4.55e+07    2.29e+07
Car                        -8.965e+06   9.32e+06     -0.962      0.336   -2.72e+07    9.31e+06
Landsize                    1.702e+04   2.13e+04      0.801      0.423   -2.47e+04    5.87e+04
BuildingArea               -4.684e+05   1.59e+05     -2.944      0.003    -7.8e+05   -1.56e+05
YearBuilt                   1.596e+06   3.51e+05      4.552      0.000    9.08e+05    2.28e+06
Lattitude                  -1.495e+08   1.12e+08     -1.339      0.181   -3.68e+08    6.94e+07
Longtitude                 -5.413e+07   2.95e+07     -1.834      0.067   -1.12e+08    3.72e+06
Propertycount              -6176.6475   2110.101     -2.927      0.003   -1.03e+04   -2039.827
Rooms^2                    -1.623e+04    1.6e+04     -1.012      0.311   -4.77e+04    1.52e+04
Rooms Distance             -2.252e+04   5636.830     -3.995      0.000   -3.36e+04   -1.15e+04
Rooms Postcode              1726.4942    577.899      2.988      0.003     593.532    2859.457
Rooms Bedroom2             -1.538e+04   2.05e+04     -0.750      0.453   -5.56e+04    2.48e+04
Rooms Bathroom              4.938e+04   2.02e+04      2.451      0.014    9877.075    8.89e+04
Rooms Car                    1.93e+04   1.78e+04      1.084      0.278   -1.56e+04    5.42e+04
Rooms Landsize               176.4874     53.538      3.296      0.001      71.526     281.448
Rooms BuildingArea          -471.0399    160.057     -2.943      0.003    -784.829    -157.251
Rooms YearBuilt              432.8089    474.300      0.913      0.362    -497.050    1362.667
Rooms Lattitude             6.811e+05   3.76e+05      1.810      0.070   -5.67e+04    1.42e+06
Rooms Longtitude            -2.76e+05   3.17e+05     -0.871      0.384   -8.97e+05    3.45e+05
Rooms Propertycount           -4.0333      4.233     -0.953      0.341     -12.331       4.265
Distance^2                  1199.2846    160.423      7.476      0.000     884.777    1513.792
Distance Postcode            -15.7970     14.558     -1.085      0.278     -44.337      12.743
Distance Bedroom2           1.875e+04   5618.027      3.337      0.001    7731.212    2.98e+04
Distance Bathroom          -6994.0808   2183.786     -3.203      0.001   -1.13e+04   -2712.803
Distance Car               -4240.5686   1254.286     -3.381      0.001   -6699.577   -1781.560
Distance Landsize             -0.7908      

In [14]:
X_train_rooms = X_train[['Rooms', 'Rooms']]
X_train_rooms.columns = ['r1', 'r2']

X_train_rooms['constant'] = 1

ols_rooms = sm.OLS(y_train, X_train_rooms)

ols_rooms = ols_rooms.fit()

ols_rooms.summary()

# Run PCA on room and bedrooms. Collinearity when discarding features.
# Discarding features with high p value

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     930.3
Date:                Sun, 15 Feb 2026   Prob (F-statistic):               0.00
Time:                        15:37:46   Log-Likelihood:                -68029.
No. Observations:                4647   AIC:                         1.361e+05
Df Residuals:                    4644   BIC:                         1.361e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
r1          2.433e+16   6.19e+17      0.039      0.969   -1.19e+18    1.24e+18
r2         -2.433e+16   6.19e+17     -0.039      0.969   -1.24e+18    1.19e+18
constant    5464.8922   2.58e+04      0.212      0.832   -4.51e+04     5.6e+04
==============================================================================
Omnibus:                     2388.030   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32598.807
Skew:                           2.126   Prob(JB):                         0.00
Kurtosis:                      15.259   Cond. No.                     4.85e+14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.97e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [15]:
# Evaluate OLS on test data
y_pred_ols_poly = ols_poly.predict(X_test_poly)

print(f"OLS R2 Score: {r2_score(y_test, y_pred_ols_poly):.4f}")
print(f"OLS MAE: {root_mean_squared_error(y_test, y_pred_ols_poly):.2f}")

# Plot OLS Results
fig_ols = px.scatter(x=y_pred_ols_poly, y=y_test, labels={'x': 'Predicted Price', 'y': 'Actual Price'}, title='Predicted vs Actual Prices (Polynomial)')
fig_ols.show()

OLS R2 Score: 0.2675
OLS MAE: 631477.92


### Diagnosing Extreme Predictions
When you see extreme predictions (like very large negative numbers for prices), it's usually a sign of **overfitting** or **multicollinearity**. 

With 104 features and no regularization, OLS can become very unstable. Here is how to diagnose it:

1. **Find the outlier:** Which row is getting that negative prediction?
2. **Check the coefficients:** Are some weights suspiciously large?
3. **Examine Feature Scales:** Polynomial terms (like $Landsize^2$) can reach massive values, confusing the model.

In [16]:
# 1. Find the most extreme prediction
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred_ols_poly
})
results_df['Error'] = results_df['Actual'] - results_df['Predicted']

print("Most negative predictions:")
print(results_df.sort_values('Predicted').head(5))

Most negative predictions:
          Actual     Predicted         Error
1588   2608000.0 -1.573983e+07  1.834783e+07
11504   345000.0  3.513761e+03  3.414862e+05
3750    352500.0  2.445983e+04  3.280402e+05
8811    170000.0  4.067163e+04  1.293284e+05
8081    333000.0  6.435000e+04  2.686500e+05


In [17]:
# 2. Identify which features are driving the extreme values for that row
worst_row_idx = results_df['Predicted'].idxmin()
worst_features = X_test_poly.loc[worst_row_idx]

# Look at the top contributing features (Coefficient * Value)
contributions = (ols_poly.params * worst_features).sort_values(key=abs, ascending=False)
print(f"\nTop feature contributions for the most negative prediction (Index {worst_row_idx}):")
print(contributions.head(10))


Top feature contributions for the most negative prediction (Index 1588):
Longtitude^2            8.479171e+09
Longtitude             -7.852856e+09
Lattitude Longtitude   -7.582555e+09
Lattitude               5.655706e+09
YearBuilt               3.063927e+09
YearBuilt Longtitude   -2.284084e+09
BuildingArea           -1.457789e+09
Lattitude^2             1.255980e+09
Postcode               -1.068950e+09
Postcode Longtitude     9.474240e+08
dtype: float64


In [18]:
# 3. Check for extreme coefficients
print("\nLargest Coefficients (Potential Overfitting/Multicollinearity):")
print(ols_poly.params.sort_values(key=abs, ascending=False).head(10))


Largest Coefficients (Potential Overfitting/Multicollinearity):
Lattitude              -1.494542e+08
Bedroom2               -6.865831e+07
Rooms                   5.989929e+07
Longtitude             -5.413377e+07
Bathroom               -1.131199e+07
Car                    -8.965211e+06
Distance                8.857631e+06
YearBuilt               1.595796e+06
Lattitude Longtitude    1.381267e+06
Lattitude^2             8.770518e+05
dtype: float64


### Comparing Models with Cross-Validation
Now we can use Cross-Validation to compare our unstable OLS model against regularized versions (Ridge and Lasso).

#### OLS: Statsmodels vs. Scikit-learn
A common misconception is that `LinearRegression` in scikit-learn uses Gradient Descent. In fact, it uses **Ordinary Least Squares (OLS)** via a mathematical technique called Singular Value Decomposition (SVD), which is a "closed-form" solution—just like `statsmodels`.

If you want to use Gradient Descent in scikit-learn, you would use `SGDRegressor`.

#### Do Ridge and Lasso use Gradient Descent?
By default, **no**. Scikit-learn uses different optimization strategies for them:

*   **Ridge:** Usually uses a **closed-form solution** (via Cholesky or SVD), similar to `LinearRegression`. It only uses gradient-based methods (like SAG or SAGA) if the dataset is very large or if you explicitly request it via the `solver` parameter.
*   **Lasso:** Uses **Coordinate Descent**. This is an iterative algorithm that optimizes one coefficient at a time. Standard Gradient Descent isn't used because the L1 penalty is not differentiable at zero (it has a "v-shape" at the origin).

**To use Gradient Descent for these:**
Use `SGDRegressor` and set the `penalty` parameter:
*   `SGDRegressor(penalty='l2')` is the Gradient Descent version of **Ridge**.
*   `SGDRegressor(penalty='l1')` is the Gradient Descent version of **Lasso**.

The most elegant way to compare them is to put them both in the same cross-validation loop. Since they use the same underlying math, they should produce nearly identical results.

**Crucial Step: Scaling**
Regularization penalizes large coefficients. If one feature is measured in millions (like $Landsize^2$) and another in units (like $Rooms$), the model will unfairly penalize the $Rooms$ coefficient. We use `StandardScaler` to put them all on the same scale.

**Preventing Data Leakage:**
By putting the `StandardScaler` inside a `Pipeline`, cross-validation will:
1. Fit the scaler on the 4 training folds.
2. Transform the 4 training folds.
3. Fit the model.
4. Transform the 1 validation fold using the scaler from step 1.
5. Score the model.

This prevents information from the validation fold from "leaking" into the training process.


In [19]:
# To include statsmodels in our comparison, we create a simple wrapper
# so it works with the scikit-learn Pipeline and cross_validate.
class StatsmodelsOLS(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        # statsmodels OLS requires an explicit constant (intercept)
        X_with_const = sm.add_constant(X, has_constant='add')
        self.model_ = sm.OLS(y, X_with_const)
        self.results_ = self.model_.fit()
        return self

    def predict(self, X):
        X_with_const = sm.add_constant(X, has_constant='add')
        return self.results_.predict(X_with_const)

# Define our candidates
# We include:
# 1. Statsmodels OLS (Benchmark)
# 2. Sklearn LinearRegression (Equivalent to Statsmodels, uses SVD)
# 3. Sklearn SGDRegressor (Uses Gradient Descent)
# 4. Regularized models (Ridge/Lasso)
models = {
    "Statsmodels OLS": StatsmodelsOLS(),
    "Sklearn LinearRegression": LinearRegression(),
    # "Sklearn SGDRegressor (GD)": SGDRegressor(max_iter=10000, tol=1e-4, random_state=0),
    "Ridge (alpha=10)": Ridge(alpha=10),
    "Lasso (alpha=100)": Lasso(alpha=100, max_iter=10000)
}

cv_results_list = []

for name, model in models.items():
    # Create a pipeline: Scale -> Model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])
    
    # Perform 5-fold Cross-Validation on the training data
    scores = cross_validate(
        pipeline, 
        X_train_poly, 
        y_train, 
        cv=5, 
        scoring='neg_root_mean_squared_error',
        return_train_score=True
    )
    
    cv_results_list.append({
        "Model": name, 
        "Train RMSE": -scores['train_score'].mean(),
        "CV RMSE": -scores['test_score'].mean(),
        "CV Std": scores['test_score'].std(),
        "Train Values": -scores['train_score'],
        "CV Values": -scores['test_score']
    })

comparison_df = pd.DataFrame(cv_results_list).sort_values("CV RMSE")
comparison_df[["Model", "Train RMSE", "CV RMSE", "CV Std"]]

C:\Users\zak\Projects\PyCharmProjects\data-science\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:716: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.910e+13, tolerance: 1.557e+11

C:\Users\zak\Projects\PyCharmProjects\data-science\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:716: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.372e+13, tolerance: 1.500e+11

C:\Users\zak\Projects\PyCharmProjects\data-science\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:716: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.73

,Model,Train RMSE,CV RMSE,CV Std
2,Ridge (alpha=10),327627.422304,366899.648494,46156.826323
3,Lasso (alpha=100),315668.708753,370840.167138,41802.731386
0,Statsmodels OLS,304701.222071,418022.302490,102420.458016
1,Sklearn LinearRegression,304701.222071,418022.302490,102420.458016


### Visualizing Model Stability
The mean (average) score is only half the story. A model might have a low average error but be very **unstable**—meaning it performs great on some parts of your data and very poorly on others. 

By visualizing the distribution of scores across all cross-validation folds, we can identify:
1.  **Stability:** Does the model perform consistently across different subsets of data?
2.  **Outliers:** Are there specific folds where the model fails completely?
3.  **Reliability:** Is the "best" model significantly better, or is it just lucky on certain folds?

In the boxplot below, the **shorter the box**, the more stable the model is.

In [20]:
# We "explode" the list of scores so each fold has its own row in a new DataFrame
plot_df = pd.DataFrame(cv_results_list).explode("CV Values")
plot_df["CV Values"] = plot_df["CV Values"].astype(float)

fig = px.box(
    plot_df, 
    x="Model", 
    y="CV Values", 
    color="Model",
    title="Distribution of CV RMSE Across Folds (Lower is Better)",
    points="all", # This shows every individual fold result
    labels={"CV Values": "RMSE"}
)
fig.update_layout(showlegend=False)
fig.show()

### Final Model Selection and Evaluation
The "best" way to evaluate performance depends on your goal, but a robust pipeline usually includes:
1.  **Cross-Validation Distribution:** (Above) To ensure the model is stable and not just "lucky" on one data split.
2.  **Test Set Performance:** To see how it generalizes to truly unseen data.
3.  **Residual Analysis:** To check if the model's errors are random or if there's a pattern the model is missing (e.g., if it consistently underpredicts expensive houses).

Below, we take the best model from our comparison and perform a final check.

In [21]:
# 1. Select the best model from our CV results
best_model_name = comparison_df.iloc[0]["Model"]
print(f"Selecting {best_model_name} as the final model based on CV performance.")

# 2. Get the model instance
# (We pick the regressor that performed best in our dictionary)
if "Ridge" in best_model_name:
    final_regressor = Ridge(alpha=10)
elif "Lasso" in best_model_name:
    final_regressor = Lasso(alpha=100)
else:
    final_regressor = LinearRegression()


# 3. Fit the final pipeline on ALL training data
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', final_regressor)
])
final_pipeline.fit(X_train_poly, y_train)

# 4. Final evaluation on the Test Set
y_test_pred = final_pipeline.predict(X_test_poly)

print(f"\n--- Final Test Results ({best_model_name}) ---")
print(f"MAE: {root_mean_squared_error(y_test, y_test_pred):.2f}")
print(f"R2 Score: {r2_score(y_test, y_test_pred):.4f}")

# 5. Residual Plot
# A good model's residuals should be centered around zero with no obvious pattern.
residuals = y_test - y_test_pred
fig_res = px.scatter(
    x=y_test_pred, 
    y=residuals,
    labels={'x': 'Predicted Price', 'y': 'Residual (Actual - Predicted)'},
    title=f'Residual Plot: {best_model_name}',
    marginal_y="histogram", # Helps see if errors are normally distributed
    opacity=0.5
)
fig_res.add_hline(y=0, line_dash="dash", line_color="red")
fig_res.show()


Selecting Ridge (alpha=10) as the final model based on CV performance.

--- Final Test Results (Ridge (alpha=10)) ---
MAE: 601940.73
R2 Score: 0.3345


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, KFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor

# 1. Create a Mixed-Type Dataset
# We create 1000 rows. 'cat_feature' is text, others are numbers.
data = pd.DataFrame({
    'num_1': np.random.normal(0, 1, 1000),
    'num_2': np.random.normal(5, 2, 1000), # Informative
    'num_3': np.random.normal(-5, 2, 1000),
    'cat_1': np.random.choice(['A', 'B', 'C'], 1000), # Categorical
    'cat_2': np.random.choice(['X', 'Y'], 1000)       # Categorical
})
# Target variable depends heavily on num_2 and cat_1
y = 3 * data['num_2'] + (data['cat_1'] == 'A') * 5 + np.random.normal(0, 1, 1000)

# Introduce NaNs to test imputation
data.loc[::10, 'num_1'] = np.nan
data.loc[::10, 'cat_1'] = np.nan

# Define feature groups
numeric_features = ['num_1', 'num_2', 'num_3']
categorical_features = ['cat_1', 'cat_2']

# 2. Build the Preprocessing Pipelines
# Pipeline for Numbers: Impute Mean -> Scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Pipeline for Categories: Impute 'missing' -> OneHotEncode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine them into a single Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    verbose_feature_names_out=False # Keeps names clean (e.g. 'cat_1_A' instead of 'cat__cat_1_A')
)

# 3. Create the Master Pipeline
# Data Flow: Preprocessor (Clean/Encode) -> Selection -> Model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(mutual_info_regression, k=4)), # Select top 4 features
    ('regressor', RandomForestRegressor(random_state=42))
])

# 4. Run Cross-Validation with Inspection
cv = KFold(n_splits=5, shuffle=True, random_state=42)

results = cross_validate(
    pipeline,
    data,
    y,
    cv=cv,
    scoring='neg_mean_squared_error',
    return_train_score=True,  # Give us training scores
    return_estimator=True     # Give us the fitted pipeline for each fold
)

# 5. Extract and Display Results
print(f"{'Fold':<5} | {'Train RMSE':<10} | {'Test RMSE':<10} | {'Selected Features'}")
print("-" * 80)

for i, estimator in enumerate(results['estimator']):
    # Recover the feature names after OneHotEncoding
    # The preprocessor is the first step in our pipeline
    feature_names_out = estimator.named_steps['preprocessor'].get_feature_names_out()

    # Get the boolean mask of selected features from step 2 ('selector')
    mask = estimator.named_steps['selector'].get_support()

    # Filter the names
    selected_feats = feature_names_out[mask]

    # Calculate RMSEs
    train_rmse = np.sqrt(-results['train_score'][i])
    test_rmse = np.sqrt(-results['test_score'][i])

    print(f"{i+1:<5} | {train_rmse:.4f}     | {test_rmse:.4f}     | {list(selected_feats)}")

print("-" * 80)
print(f"Average Test RMSE: {np.sqrt(-results['test_score']).mean():.4f}")

Fold  | Train RMSE | Test RMSE  | Selected Features
--------------------------------------------------------------------------------
1     | 0.5835     | 1.5190     | ['num_2', 'num_3', 'cat_1_A', 'cat_1_B']
2     | 0.6069     | 1.5387     | ['num_2', 'cat_1_A', 'cat_2_X', 'cat_2_Y']
3     | 0.5918     | 1.5566     | ['num_2', 'cat_1_A', 'cat_1_B', 'cat_2_Y']
4     | 0.5693     | 1.4531     | ['num_2', 'num_3', 'cat_1_A', 'cat_1_B']
5     | 0.5580     | 1.7879     | ['num_2', 'cat_1_A', 'cat_1_B', 'cat_2_Y']
--------------------------------------------------------------------------------
Average Test RMSE: 1.5711
